### 목적
- LLM이 작업을 수행할 수 있도록 정보와 도구를 적절한 형식으로 제공하는 동적 시스템 구축

#### 컨텍스트 엔지니어링 vs 프롬프트 엔지니어링
| 구분    | 프롬프트 엔지니어링                 | 컨텍스트 엔지니어링                                        |
| ----- | -------------------------- | ------------------------------------------------- |
| 초점    | 한 번의 입력문 설계                | 전체 대화·상태·환경의 구조 설계                                |
| 단위    | 문자열 prompt                 | 구조화된 context object                               |
| 접근 방식 | “어떻게 물어볼까?”                | “무엇을 알고, 어떤 상황에서 대답해야 하나?”                        |
| 주요 기술 | few-shot, CoT, role prompt | context_schema, dynamic_prompt, middleware, state |
| 결과    | 즉시 품질 향상                   | 지속적 일관성, 개인화, 멀티턴 정확도                             |
| 비유    | 문장 기술                      | 시스템 설계                                            |


#### 컨텍스트 종류
| 컨텍스트 종류                | 목적                                      | 어디에 저장되나   | 지속 시간                |
| ---------------------- | --------------------------------------- | ---------- | -------------------- |
| **Model Context**      | 모델에게 주는 일시적 정보 (프롬프트, 메시지, 툴 목록 등)      | 요청 내부      | 🔹짧음 (한 번 호출용)       |
| **Tool Context**       | 툴 함수가 읽고 쓸 수 있는 환경(context/state/store) | Runtime 객체 | 🔸중간 (세션 내 유지)       |
| **Life-cycle Context** | 실행 전후 감시·제어용 (미들웨어, 요약, 검열 등)           | 미들웨어 계층    | 🔸지속 (여러 호출 간 유지 가능) |

- Model Context: 매 호출마다 새로 생성 (Transient)
- Tool/Life-cycle Context: 실행 중 혹은 장기적으로 유지 (Persistent)

#### 컨텍스트 데이터 구성
- context: 현재 사용자나 환경같은 고정 정보
    - ex) user_id, language, timezone
- state: 대화 중 잠깐 유지되는 단기 메모리
    - ex) 직전 메시지, 업로드 파일 요약
- store: 장기 메모리
    - ex) 사용자 선호, 과거 대화 요약, 영구 기록

#### 각 컨텍스트 별로 다루는 데이터 소스
| 컨트롤 계층                 | 읽는 데이터                      | 쓰는 데이터          | 주요 접근 방식                                                 | 비고          |
| ---------------------- | --------------------------- | --------------- | -------------------------------------------------------- | ----------- |
| **Model Context**      | `context`, `state`, `store` | ❌ (읽기 중심)       | `request.context`, `request.state`, `request.store`      | 모델 호출 직전/직후 |
| **Tool Context**       | `state`, `store`            | ✅ (Command로 갱신) | `runtime.state`, `runtime.store`                         | 툴 함수 내 실행   |
| **Life-cycle Context** | `state`, `store`            | ✅ (요약/기록)       | `before_model`, `after_model`, `SummarizationMiddleware` | 대화 관리/압축    |


#### Model Context 조작 기법 - System prompt
- @dynamic_prompt
    - 실행 시점에 state, runtime.context, runtime.store를 읽어 system prompt를 생성

In [ ]:
# state 사용 예시 코드
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

@dynamic_prompt
def system_prompt_selector(request: ModelRequest) -> str:
    """
    현재 대화 길이에 따라 system prompt를 자동 변경한다.
    - 메시지가 많으면: 간결하게
    - 메시지가 적으면: 일반 모드
    """
    # request.state["messages"] 를 request.messages 로 줄여서 적을 수 있다. -> 최신 버전 권장은 축약 형태
    # message_count = len(request.messages)
    message_count = request.state["messages"]
    print(len(request.state["messages"]))
    base = "너는 유용한 어시스턴트이다."
    if message_count > 5:
        base += "\n대화가 길어졌으니 간결하게 답변해라."
    else:
        base += "\n상세하고 친절하게 설명해라."
    return base

agent = create_agent(
    model=llm,
    tools=[],  
    middleware=[system_prompt_selector]
)

conversation_short = {"messages": [{"role": "user", "content": "인공지능이 뭐야?"}]}
conversation_long = {"messages": [{"role": "user", "content": "인공지능이 뭐야?"} for i in range(7)]}

print("▶ 짧은 대화:")
result_short = agent.invoke(conversation_short)
print(result_short["messages"][-1].content, "\n")

print("▶ 긴 대화:")
result_long = agent.invoke(conversation_long)
print(result_long["messages"][-1].content)

▶ 짧은 대화:
1
인공지능(Artificial Intelligence, AI)은 기계나 컴퓨터 시스템이 인간처럼 사고하고 학습하는 능력을 갖추도록 하는 기술을 의미합니다. AI는 다양한 분야에서 활용되며, 주로 사람의 인지 기능을 모방하는 프로그램과 시스템을 만드는 데 중점을 둡니다. 아래에 인공지능의 주요 개념과 분류를 설명해 드리겠습니다.

### 1. 인공지능의 범주
인공지능은 크게 세 가지 범주로 나눌 수 있습니다:

- **약한 인공지능(Weak AI)**: 특정 작업이나 문제를 해결하기 위해 디자인된 AI입니다. 예를 들어, 음성 인식 프로그램이나 추천 시스템이 해당됩니다. 이들은 특정한 문제에 대해서만 능력을 발휘하며, 일반적인 지능을 가지지는 않습니다.

- **강한 인공지능(Strong AI)**: 인간과 같은 수준의 지능을 가진 AI를 의미합니다. 이 AI는 다양한 분야에서 학습하고 문제를 해결할 수 있으며, 인간처럼 사고하고 이해할 수 있는 능력을 가질 것으로 기대됩니다. 현재는 강한 인공지능이 실현되지 않았습니다.

- **초인공지능(Superintelligence)**: 인간 지능을 넘어서는 AI로, 이론적으로 상상할 수 있는 개념입니다. 초인공지능은 인간의 능력을 뛰어넘는 복잡한 문제 해결이 가능할 것으로 예상됩니다.

### 2. 인공지능 기술
AI를 구현하는 데 사용되는 다양한 기술이 있습니다:

- **머신 러닝(Machine Learning)**: 기계가 데이터를 통해 학습하고, 경험을 쌓아 가는 방법입니다. 예를 들어, 이미지 인식 시스템은 많은 이미지를 분석하여 객체를 인식할 수 있게 됩니다.

- **딥 러닝(Deep Learning)**: 머신 러닝의 하위 분야로, 인공 신경망을 기반으로 한 학습 방식입니다. 이는 복잡한 패턴을 인식하고 처리하는 데 매우 효과적입니다.

- **자연어 처리(Natural Language Processing, NLP)**: 인간의 언어를 이해하고 해석하는 기술로, 음성 인식, 번역, 텍스

In [19]:
# runtime.context와 runtime.store 사용 예시 코드

import os
from dataclasses import dataclass
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langgraph.store.memory import InMemoryStore

load_dotenv()

@dataclass
class Context:
    user_id: str

# 장기 스토어 준비
store = InMemoryStore()
store.put(("preferences",), "u-001", {"communication_style": "반말"})

# 동적 시스템 프롬프트: context.user_id로 store 조회
@dynamic_prompt
def store_aware_prompt(request: ModelRequest) -> str:
    user_id = request.runtime.context.user_id # 런타임에서 context로 들어온 user_id
    print("user id:", user_id)
    item = request.runtime.store.get(("preferences",), user_id) # 장기 메모리에서 user_id로 선호 검색
    base = "너는 유용한 조수이다."
    if item:
        style = item.value.get("communication_style", "균형잡힌 어투")
        base += f"\n유저의 선호도는 {style} 이다."
    return base

agent = create_agent(
    model=llm,
    tools=[],
    middleware=[store_aware_prompt],
    context_schema=Context,
    store=store,
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "간단히 요약해서 인공지능 정의해줘"}]},
    context=Context(user_id="u-001"),
)

print(result["messages"][-1].content)

user id: u-001
인공지능은 인간의 지능을 모방하거나 대체할 수 있는 컴퓨터 시스템이나 프로그램이야. 주로 학습, 문제 해결, 패턴 인식 같은 기능을 통해 데이터에서 통찰을 얻고 의사 결정을 내리는 데 사용돼.


#### Model Context 조작 기법 - Messages
- @wrap_model_call
    - ㄴ